In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform
import time, os, datetime
from Model import Residual_Unit
from Model import Attention_Block
from Model import Attention_Block_NAL
from Model import AttentionResNet92
from Model import AttentionResNet92_NAL

In [2]:
print("TF version: ",tf.__version__)
print("Keras version:",tf.keras.__version__)

TF version:  2.0.0
Keras version: 2.2.4-tf


In [3]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,TensorBoard, LearningRateScheduler
from tensorflow.keras.models import load_model

In [5]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:45000, :, :, :]
y_train = y_train[:45000]
x_val = x_train[-5000:, :, :, :]
y_val = y_train[-5000:]

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (45000, 32, 32, 3)
y_train shape: (45000, 10)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 10)


In [6]:
# define generators for training and validation data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# compute quantities required for feature normalization
train_datagen.fit(x_train)
val_datagen.fit(x_val)
test_datagen.fit(x_test)

## Training Residual AttentionNet 92

In [7]:
model = AttentionResNet92(shape=(32,32,3), in_channel=32, kernel_size=5, n_classes=10, dropout=0.4, regularization=0.00001)

def lr_schedule(epoch):
    lr = 1e-5
    if epoch > 80:
        lr *= 1e-5
    elif epoch > 60:
        lr *= 1e-4
    elif epoch >40:
        lr *= 1e-2
    print('Learning rate:', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# define loss, metrics, optimizer
optimizer = Adam(lr = lr_schedule(0) )
#optimizer = SGD(lr = lr_schedule(0), momentum=0.9, nesterov = True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Learning rate: 1e-05
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 32)   2432        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 32)   128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 32)   0           batch_normalization[0][0]        
_________________________________________________________________________

__________________________________________________________________________________________________
activation_130 (Activation)     (None, 4, 4, 512)    0           batch_normalization_124[0][0]    
__________________________________________________________________________________________________
conv2d_177 (Conv2D)             (None, 4, 4, 512)    262656      activation_130[0][0]             
__________________________________________________________________________________________________
batch_normalization_125 (BatchN (None, 4, 4, 512)    2048        conv2d_177[0][0]                 
__________________________________________________________________________________________________
activation_131 (Activation)     (None, 4, 4, 512)    0           batch_normalization_125[0][0]    
__________________________________________________________________________________________________
conv2d_178 (Conv2D)             (None, 4, 4, 512)    2359808     activation_131[0][0]             
__________

In [8]:
batch_size = 128
epc = 100

start = time.time()

# training
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_train = train_generator.n // train_generator.batch_size

# validation
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)
step_size_val = val_generator.n // val_generator.batch_size

# test
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
step_size_test = test_generator.n // test_generator.batch_size

# usefull callbacks
log_dir="Logs/92full"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)

model.fit_generator(train_generator,
                    steps_per_epoch = step_size_train,
                    epochs = epc,
                    validation_data = val_generator,
                    validation_steps = step_size_val,
                    callbacks=[tensorboard_callback, lr_reducer, lr_scheduler, early_stopper])

end = time.time()
print("Time taken by above cell is {}.".format((end-start)/60))

Learning rate: 1e-05
Epoch 1/100
351/351 [==============================] - 353s 1s/step - loss: 4.0257 - accuracy: 0.2152 - val_loss: 1.9502 - val_accuracy: 0.2965
Learning rate: 1e-05
Epoch 2/100
351/351 [==============================] - 349s 993ms/step - loss: 2.3915 - accuracy: 0.2608 - val_loss: 1.9061 - val_accuracy: 0.3431
Learning rate: 1e-05
Epoch 3/100
351/351 [==============================] - 350s 996ms/step - loss: 2.1082 - accuracy: 0.2849 - val_loss: 1.8864 - val_accuracy: 0.3516
Learning rate: 1e-05
Epoch 4/100
351/351 [==============================] - 348s 991ms/step - loss: 1.9638 - accuracy: 0.3173 - val_loss: 1.7730 - val_accuracy: 0.3802
Learning rate: 1e-05
Epoch 5/100
351/351 [==============================] - 349s 994ms/step - loss: 1.8782 - accuracy: 0.3354 - val_loss: 1.7585 - val_accuracy: 0.3876
Learning rate: 1e-05
Epoch 6/100
351/351 [==============================] - 347s 990ms/step - loss: 1.8130 - accuracy: 0.3539 - val_loss: 1.6809 - val_accuracy: 0.

Learning rate: 1.0000000000000001e-07
Epoch 49/100
351/351 [==============================] - 343s 976ms/step - loss: 1.0336 - accuracy: 0.6347 - val_loss: 0.9399 - val_accuracy: 0.6647
Learning rate: 1.0000000000000001e-07
Epoch 50/100
351/351 [==============================] - 343s 976ms/step - loss: 1.0259 - accuracy: 0.6377 - val_loss: 0.9391 - val_accuracy: 0.6647
Learning rate: 1.0000000000000001e-07
Epoch 51/100
351/351 [==============================] - 341s 972ms/step - loss: 1.0339 - accuracy: 0.6345 - val_loss: 0.9373 - val_accuracy: 0.6665
Learning rate: 1.0000000000000001e-07
Epoch 52/100
351/351 [==============================] - 342s 974ms/step - loss: 1.0295 - accuracy: 0.6349 - val_loss: 0.9372 - val_accuracy: 0.6663
Learning rate: 1.0000000000000001e-07
Epoch 53/100
351/351 [==============================] - 341s 970ms/step - loss: 1.0337 - accuracy: 0.6349 - val_loss: 0.9339 - val_accuracy: 0.6671
Learning rate: 1.0000000000000001e-07
Epoch 54/100
351/351 [==========

In [9]:
# evaluation
val_scores = model.evaluate_generator(val_generator, verbose=0)
test_scores = model.evaluate_generator(test_generator, verbose=0)
print('validation loss:', val_scores[0])
print('validation accuracy:', val_scores[1])
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

validation loss: 0.9397639229893684
validation accuracy: 0.668
Test loss: 1.018264941776855
Test accuracy: 0.6472


## Training Residual AttentionNet 92 - NAL

In [6]:
model = AttentionResNet92_NAL(shape=(32,32,3), in_channel=32, kernel_size=5, n_classes=10, dropout=None, regularization=0.00001)

def lr_schedule(epoch):
    lr = 1e-5
    if epoch > 80:
        lr *= 1e-5
    elif epoch > 60:
        lr *= 1e-4
    elif epoch >40:
        lr *= 1e-2
    print('Learning rate:', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)
# define loss, metrics, optimizer
optimizer = Adam(lr = lr_schedule(0) )
#optimizer = SGD(lr = lr_schedule(0), momentum=0.9, nesterov = True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

Learning rate: 1e-05


In [7]:
batch_size = 128
epc = 100

start = time.time()

# training
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_train = train_generator.n // train_generator.batch_size

# validation
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)
step_size_val = val_generator.n // val_generator.batch_size

# test
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
step_size_test = test_generator.n // test_generator.batch_size

# usefull callbacks
log_dir="Logs/92full-NAL"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', patience=15, verbose=1)

model.fit_generator(train_generator,
                    steps_per_epoch = step_size_train,
                    epochs = epc,
                    validation_data = val_generator,
                    validation_steps = step_size_val,
                    callbacks=[tensorboard_callback, lr_reducer, lr_scheduler, early_stopper])

end = time.time()
print("Time taken by above cell is {}.".format((end-start)/60))

Learning rate: 1e-05
Epoch 1/100
351/351 [==============================] - 275s 783ms/step - loss: 2.2758 - acc: 0.1572 - val_loss: 2.3754 - val_acc: 0.1446
Learning rate: 1e-05
Epoch 2/100
351/351 [==============================] - 211s 600ms/step - loss: 2.1031 - acc: 0.2203 - val_loss: 2.1311 - val_acc: 0.2378
Learning rate: 1e-05
Epoch 3/100
351/351 [==============================] - 213s 608ms/step - loss: 2.0244 - acc: 0.2472 - val_loss: 2.0767 - val_acc: 0.2604
Learning rate: 1e-05
Epoch 4/100
351/351 [==============================] - 216s 614ms/step - loss: 1.9692 - acc: 0.2716 - val_loss: 2.0920 - val_acc: 0.2658
Learning rate: 1e-05
Epoch 5/100
351/351 [==============================] - 216s 614ms/step - loss: 1.9281 - acc: 0.2881 - val_loss: 1.9620 - val_acc: 0.2955
Learning rate: 1e-05
Epoch 6/100
351/351 [==============================] - 214s 610ms/step - loss: 1.8873 - acc: 0.3038 - val_loss: 1.9176 - val_acc: 0.3215
Learning rate: 1e-05
Epoch 7/100
351/351 [==========

## Training CIFAR-100

In [10]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train[:25000, :, :, :]
y_train = y_train[:25000]
x_val = x_train[-5000:, :, :, :]
y_val = y_train[-5000:]

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 100)
y_val = to_categorical(y_val, 100)
y_test = to_categorical(y_test, 100)

print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

x_train shape: (25000, 32, 32, 3)
y_train shape: (25000, 100)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 100)


In [11]:
# define generators for training and validation data
train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

val_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

test_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)

# compute quantities required for feature normalization
train_datagen.fit(x_train)
val_datagen.fit(x_val)
test_datagen.fit(x_test)

In [12]:
model = AttentionResNet92(shape=(32,32,3), in_channel=32, kernel_size=5, n_classes=100, dropout=0.4, regularization=0.0001)

def lr_schedule(epoch):
    lr = 1e-5
    if epoch > 80:
        lr *= 1e-5
    elif epoch > 60:
        lr *= 1e-4
    elif epoch >40:
        lr *= 1e-2
    print('Learning rate:', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

# define loss, metrics, optimizer
optimizer = Adam(lr = lr_schedule(0) )
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Learning rate: 1e-05
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_193 (Conv2D)             (None, 32, 32, 32)   2432        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_136 (BatchN (None, 32, 32, 32)   128         conv2d_193[0][0]                 
__________________________________________________________________________________________________
activation_142 (Activation)     (None, 32, 32, 32)   0           batch_normalization_136[0][0]    
_______________________________________________________________________

batch_normalization_223 (BatchN (None, 2, 2, 512)    2048        conv2d_313[0][0]                 
__________________________________________________________________________________________________
activation_224 (Activation)     (None, 4, 4, 512)    0           batch_normalization_215[0][0]    
__________________________________________________________________________________________________
activation_232 (Activation)     (None, 2, 2, 512)    0           batch_normalization_223[0][0]    
__________________________________________________________________________________________________
conv2d_304 (Conv2D)             (None, 4, 4, 512)    262656      activation_224[0][0]             
__________________________________________________________________________________________________
conv2d_314 (Conv2D)             (None, 2, 2, 512)    262656      activation_232[0][0]             
__________________________________________________________________________________________________
conv2d_315

In [13]:
batch_size = 128
epc = 60

start = time.time()

# training
train_generator = train_datagen.flow(x_train, y_train, batch_size=batch_size)
step_size_train = train_generator.n // train_generator.batch_size

# validation
val_generator = val_datagen.flow(x_val, y_val, batch_size=batch_size)
step_size_val = val_generator.n // val_generator.batch_size

# test
test_generator = test_datagen.flow(x_test, y_test, batch_size=batch_size)
step_size_test = test_generator.n // test_generator.batch_size

# usefull callbacks
log_dir="Logs/92-CIFAR100"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', patience=15, verbose=1)

model.fit_generator(train_generator,
                    steps_per_epoch = step_size_train,
                    epochs = epc,
                    validation_data = val_generator,
                    validation_steps = step_size_val,
                    callbacks=[tensorboard_callback, lr_reducer, lr_scheduler, early_stopper])

end = time.time()
print("Time taken by above cell is {}.".format((end-start)/60))

Learning rate: 1e-05
Epoch 1/60
195/195 [==============================] - 201s 1s/step - loss: 8.5849 - accuracy: 0.0233 - val_loss: 4.7033 - val_accuracy: 0.0284
Learning rate: 1e-05
Epoch 2/60
195/195 [==============================] - 200s 1s/step - loss: 5.2430 - accuracy: 0.0393 - val_loss: 4.2774 - val_accuracy: 0.0689
Learning rate: 1e-05
Epoch 3/60
195/195 [==============================] - 200s 1s/step - loss: 4.6734 - accuracy: 0.0519 - val_loss: 4.1507 - val_accuracy: 0.0905
Learning rate: 1e-05
Epoch 4/60
195/195 [==============================] - 202s 1s/step - loss: 4.4405 - accuracy: 0.0653 - val_loss: 4.0260 - val_accuracy: 0.0988
Learning rate: 1e-05
Epoch 5/60
195/195 [==============================] - 201s 1s/step - loss: 4.2837 - accuracy: 0.0763 - val_loss: 3.9469 - val_accuracy: 0.1172
Learning rate: 1e-05
Epoch 6/60
195/195 [==============================] - 200s 1s/step - loss: 4.1919 - accuracy: 0.0852 - val_loss: 3.8803 - val_accuracy: 0.1290
Learning rate: 1

195/195 [==============================] - 199s 1s/step - loss: 3.7091 - accuracy: 0.1437 - val_loss: 3.4185 - val_accuracy: 0.1921
Learning rate: 1e-05
Epoch 16/60
195/195 [==============================] - 200s 1s/step - loss: 3.6745 - accuracy: 0.1503 - val_loss: 3.3412 - val_accuracy: 0.2151
Learning rate: 1e-05
Epoch 17/60
195/195 [==============================] - 200s 1s/step - loss: 3.6354 - accuracy: 0.1572 - val_loss: 3.2948 - val_accuracy: 0.2179
Learning rate: 1e-05
Epoch 18/60
195/195 [==============================] - 198s 1s/step - loss: 3.5996 - accuracy: 0.1610 - val_loss: 3.2697 - val_accuracy: 0.2238
Learning rate: 1e-05
Epoch 19/60
195/195 [==============================] - 199s 1s/step - loss: 3.5681 - accuracy: 0.1628 - val_loss: 3.2217 - val_accuracy: 0.2338
Learning rate: 1e-05
Epoch 20/60
195/195 [==============================] - 200s 1s/step - loss: 3.5194 - accuracy: 0.1696 - val_loss: 3.2425 - val_accuracy: 0.2222
Learning rate: 1e-05
Epoch 21/60
195/195 [=

195/195 [==============================] - 199s 1s/step - loss: 3.2632 - accuracy: 0.2121 - val_loss: 2.9424 - val_accuracy: 0.2776
Learning rate: 1e-05
Epoch 31/60
195/195 [==============================] - 200s 1s/step - loss: 3.2297 - accuracy: 0.2182 - val_loss: 2.9310 - val_accuracy: 0.2810
Learning rate: 1e-05
Epoch 32/60
195/195 [==============================] - 197s 1s/step - loss: 3.1935 - accuracy: 0.2291 - val_loss: 2.8797 - val_accuracy: 0.2899
Learning rate: 1e-05
Epoch 33/60
195/195 [==============================] - 199s 1s/step - loss: 3.1723 - accuracy: 0.2300 - val_loss: 2.8209 - val_accuracy: 0.3007
Learning rate: 1e-05
Epoch 34/60
195/195 [==============================] - 202s 1s/step - loss: 3.1444 - accuracy: 0.2336 - val_loss: 2.7806 - val_accuracy: 0.3107
Learning rate: 1e-05
Epoch 35/60
195/195 [==============================] - 203s 1s/step - loss: 3.1098 - accuracy: 0.2414 - val_loss: 2.7812 - val_accuracy: 0.3059
Learning rate: 1e-05
Epoch 36/60
195/195 [=

195/195 [==============================] - 201s 1s/step - loss: 2.8878 - accuracy: 0.2791 - val_loss: 2.5838 - val_accuracy: 0.3496
Learning rate: 1.0000000000000001e-07
Epoch 46/60
195/195 [==============================] - 201s 1s/step - loss: 2.8828 - accuracy: 0.2866 - val_loss: 2.5796 - val_accuracy: 0.3502
Learning rate: 1.0000000000000001e-07
Epoch 47/60
195/195 [==============================] - 204s 1s/step - loss: 2.8845 - accuracy: 0.2816 - val_loss: 2.5771 - val_accuracy: 0.3512
Learning rate: 1.0000000000000001e-07
Epoch 48/60
195/195 [==============================] - 206s 1s/step - loss: 2.8846 - accuracy: 0.2813 - val_loss: 2.5760 - val_accuracy: 0.3510
Learning rate: 1.0000000000000001e-07
Epoch 49/60
195/195 [==============================] - 206s 1s/step - loss: 2.8858 - accuracy: 0.2828 - val_loss: 2.5745 - val_accuracy: 0.3518
Learning rate: 1.0000000000000001e-07
Epoch 50/60
195/195 [==============================] - 204s 1s/step - loss: 2.8785 - accuracy: 0.2858 

195/195 [==============================] - 203s 1s/step - loss: 2.8744 - accuracy: 0.2852 - val_loss: 2.5639 - val_accuracy: 0.3556
Learning rate: 1.0000000000000001e-07
Epoch 60/60
195/195 [==============================] - 205s 1s/step - loss: 2.8677 - accuracy: 0.2867 - val_loss: 2.5591 - val_accuracy: 0.3556
Time taken by above cell is 201.2063398520152.


In [14]:
# evaluation
val_scores = model.evaluate_generator(val_generator, verbose=0)
test_scores = model.evaluate_generator(test_generator, verbose=0)
print('validation loss:', val_scores[0])
print('validation accuracy:', val_scores[1])
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

validation loss: 2.525966650247574
validation accuracy: 0.356
Test loss: 2.8661425928526287
Test accuracy: 0.3053
